In [3]:
import pandas as pd

df_full = pd.read_csv('data\\steam_reviews.csv')

df = df_full[:500000]

#app_names = df_sample.groupby('app_name')
#print(app_names)

In [4]:
# Get number of reviews per game
app_names = df.groupby('app_name')['app_name'].value_counts()
print(app_names)

app_name
18 Wheels of Steel: American Long Haul      165
3D Ultra Minigolf Adventures Deluxe          37
7 Wonders II                                 26
7 Wonders: The Treasures of Seven            20
9th Company - Roots of Terror                72
                                          ...  
Worms Revolution                           1478
XCOM: Enemy Unknown                       19654
Zero Gear                                    45
iBomber Defense                              97
inMomentum                                   95
Name: count, Length: 319, dtype: int64


In [5]:
recommended = df.groupby('review_score')['review_score'].value_counts()
print(recommended)

def strLen(text):
    return len(str(text))

# kör denna efter texten bearbetats?
df['review_length'] = df['review_text'].apply(strLen)

mean_len = df.groupby('review_score')['review_length'].mean()
print(mean_len)


review_score
-1     66819
 1    433181
Name: count, dtype: int64
review_score
-1    353.943669
 1    261.358792
Name: review_length, dtype: float64



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# Add data visualization
# add wordcloud after stemming?

In [6]:
import re
import string

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    #text = re.sub('\[.*?\]', '', text)
    #text = re.sub('https?://\S+|www\.\S+', '', text)
    #text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

df['review_clean'] = df['review_text'].apply(clean_text)

#print(df_sample.iloc[5])


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
import stopwords
stop_words = stopwords.get_stopwords('english')
#stop_words = stopwords.words('english')

def remove_stopwords(text):
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    return text

df['review_clean'] = df['review_clean'].apply(remove_stopwords)




A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
import nltk

stemmer = nltk.SnowballStemmer('english')

def stemm_text(text):
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    return text

df['review_clean'] = df['review_clean'].apply(stemm_text)



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
def targetEncoded(toEncode):
    return 1 if toEncode == 1 else 0

df['review_score_target'] = df['review_score'].apply(targetEncoded)

x = df['review_clean']
y = df['review_score_target']

print(len(x), len(y))

500000 500000



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))


375000 375000
125000 125000


In [11]:

from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(stop_words=None, ngram_range=(1, 1), max_features=100)
vect.fit(x_train) # Creates a dictionary of all the words and maps each word in the output matrix

x_train_dtm = vect.transform(x_train) # Transforms the dataset to the vect dictonary
x_test_dtm = vect.transform(x_test) # Transforms the dataset to the vect dictonary


In [12]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()

tfidf_transformer.fit(x_train_dtm)
x_train_tfidf = tfidf_transformer.transform(x_train_dtm)

#x_train_tfidf

In [40]:
# Get weights of words
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer()
tfidf_matrix = vect.fit_transform(x_train[:1000])
df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names_out())
print(df)

     abail  abandon  abe  abil  abject       abl  abort  abound  about  \
0      0.0      0.0  0.0   0.0     0.0  0.000000    0.0     0.0    0.0   
1      0.0      0.0  0.0   0.0     0.0  0.000000    0.0     0.0    0.0   
2      0.0      0.0  0.0   0.0     0.0  0.000000    0.0     0.0    0.0   
3      0.0      0.0  0.0   0.0     0.0  0.000000    0.0     0.0    0.0   
4      0.0      0.0  0.0   0.0     0.0  0.000000    0.0     0.0    0.0   
..     ...      ...  ...   ...     ...       ...    ...     ...    ...   
995    0.0      0.0  0.0   0.0     0.0  0.000000    0.0     0.0    0.0   
996    0.0      0.0  0.0   0.0     0.0  0.000000    0.0     0.0    0.0   
997    0.0      0.0  0.0   0.0     0.0  0.000000    0.0     0.0    0.0   
998    0.0      0.0  0.0   0.0     0.0  0.237555    0.0     0.0    0.0   
999    0.0      0.0  0.0   0.0     0.0  0.000000    0.0     0.0    0.0   

     abrupt  ...  zambi  zelda  zeldaroguelik  zerog  zip  zombi  zone  \
0       0.0  ...    0.0    0.0       

In [13]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

# Train the model
nb.fit(x_train_dtm, y_train)

y_pred_class = nb.predict(x_test_dtm)
y_pred_prob = nb.predict_proba(x_test_dtm)[:, 1]
print(y_pred_prob)

[0.98719733 0.45785863 0.8969804  ... 0.86328383 0.97517055 0.88208671]


In [14]:
from sklearn import metrics
from sklearn.metrics import classification_report
print(metrics.accuracy_score(y_test, y_pred_class))
print(classification_report(y_test, y_pred_class))
metrics.roc_auc_score(y_test, y_pred_prob)

0.871664
              precision    recall  f1-score   support

           0       0.54      0.24      0.33     16673
           1       0.89      0.97      0.93    108327

    accuracy                           0.87    125000
   macro avg       0.72      0.60      0.63    125000
weighted avg       0.85      0.87      0.85    125000



0.796004679317431

In [18]:
import shap
#shap.initjs()
x_test_dtm_sample = shap.sample(x_train_dtm, 10000)
explainer = shap.KernelExplainer(nb.predict, x_test_dtm_sample)
shap_values = explainer.shap_values(x_test_dtm_sample)
shap.summary_plot(shap_values, x_test_dtm_sample)

Using 10000 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
  0%|          | 0/10000 [00:31<?, ?it/s]


KeyboardInterrupt: 

In [1]:
import numpy as np
print(np.__version__)

1.24.0
